# Create OCEAN Data NFT
Quickly create OCEAN Data NFTs by querying blockchain data from Dune and Flipside.

**Steps:**
1. Get Data
2. Store Data
3. Create Data NFT

In [1]:
import json
import pandas as pd
import os
import time
from dotenv import load_dotenv
import requests
import subprocess
from flipside import Flipside
from ocean_lib.web3_internal.utils import connect_to_network
from ocean_lib.example_config import get_config_dict
from ocean_lib.ocean.ocean import Ocean
from brownie.network import accounts, chain

# Load variables from .env file
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
FLIPSIDE_API_KEY = os.getenv("FLIPSIDE_API_KEY")

## 1. Get Data
### Flipside

* SDK Docs: https://docs.flipsidecrypto.com/flipside-api/get-started

In [2]:
# Query Flipside using their Python SDK
def query_flipside(sql):
    flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")
    results = flipside.query(sql)
    results_df = pd.DataFrame(results.rows, columns=results.columns).drop(
        columns=["__row_index"]
    )
    return results_df

In [3]:
sql = """
SELECT
  date(block_timestamp) as dt,
  count(distinct tx_hash) as tx_ct
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
order by 1 asc
"""
results_df = query_flipside(sql)
results_df

,dt,tx_ct
0,2023-08-28T00:00:00.000Z,385184
1,2023-08-29T00:00:00.000Z,1019692
2,2023-08-30T00:00:00.000Z,1027316
3,2023-08-31T00:00:00.000Z,1010128
4,2023-09-01T00:00:00.000Z,1006635
5,2023-09-02T00:00:00.000Z,1024172
6,2023-09-03T00:00:00.000Z,1038128
7,2023-09-04T00:00:00.000Z,583671


### Dune

* API Docs: https://dune.com/docs/api/

In [6]:
# Query Dune Analytics via API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/{q}/results?api_key={DUNE_API_KEY}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [7]:
dune_query_id = 2847682  # Query URL: https://dune.com/queries/2847682
results_df = query_dune(dune_query_id)
results_df

KeyError: 'result'

## 2. Store Data
### Add File to GitHub

*Note: This is a quick & easy solution to host the data, but can also add file to IPFS, Arweave, GCP, AWS, etc...*

In [8]:
# Create File from Query results
results_df.to_csv("query_results.csv", index=False)

In [9]:
# Store file in GitHub repo
def git_add_commit_push(commit_message):
    try:
        subprocess.run(["git", "add", "."])
        subprocess.run(["git", "commit", "-m", commit_message])
        subprocess.run(["git", "push"])
        print("Git add, commit, and push successful!")
    except Exception as e:
        print("An error occurred:", e)


# Replace 'Your commit message here' with your desired commit message
commit_message = "Your commit message here"

git_add_commit_push(commit_message)

[main c98f2ec] Your commit message here
 3 files changed, 116 insertions(+), 20 deletions(-)
 delete mode 100644 .env.example
Git add, commit, and push successful!


To https://github.com/Demoniarc/ocean_data_nfts
   6434065..c98f2ec  main -> main


## 3. Create Data NFT
### Add Data NFT to the OCEAN Protocol Marketplace

* Marketplace: https://market.oceanprotocol.com/
* SDK Docs: https://docs.oceanprotocol.com/developers/ocean.py/install

In [12]:
# Connect to Blockchain
connect_to_network("polygon-main")
config = get_config_dict("polygon-main")
ocean = Ocean(config)

ConnectionError: Already connected to network 'polygon-main'

In [13]:
# Connect to Wallet
accounts.clear()
private_key = os.getenv("PRIVATE_KEY")
wallet = accounts.add(private_key)
print("MATIC Balance:", wallet.balance() / 1e18)

Error: Non-hexadecimal digit found

In [ ]:
# Create Data NFT
name = "Data NFT (Dune)"

# Link to File
result = subprocess.run(
    ["git", "config", "--get", "user.name"], stdout=subprocess.PIPE, text=True
)
github_username = result.stdout.strip().replace(" ", "")
url = f"https://raw.githubusercontent.com/{github_username}/ocean_data_nfts/main/query_results.csv"

# Set Gas Fees
priority_fee = chain.priority_fee
max_fee = (chain.base_fee * 2) + priority_fee
gas_limit = int((priority_fee + max_fee) / 1e5)
print("priority_fee:", priority_fee, "max_fee:", max_fee, "gas_limit:", gas_limit)

# Configure Transaction Variables
tx_dict = {
    "from": wallet,
    "priority_fee": priority_fee,
    "max_fee": max_fee,
    "gas_limit": gas_limit,
}

# Publish Data NFT
(data_nft, datatoken, ddo) = ocean.assets.create_url_asset(name, url, tx_dict)

time.sleep(30)

# Set Price to Free
datatoken.create_dispenser(tx_dict)

print("Data NFT published:")
print(f"  data_nft: symbol={data_nft.symbol()}, address={data_nft.address}")
print(f"  datatoken: symbol={datatoken.symbol()}, address={datatoken.address}")
print(f"  did={ddo.did}")
print(f"  Link: https://market.oceanprotocol.com/asset/{ddo.did}")